### RNN Assignment

import needed libraries

In [65]:
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from keras.layers import GRU, Dropout
from keras.models import Sequential
from keras.layers import LSTM, SimpleRNN, Dense
import keras

import data

get max length

In [66]:
max_len=0
all_data_x=[]
all_data_y=[]
for subject_id in range(1, 11):
    file_name = f"mHealth_subject{subject_id}.log"
    
    with open(file_name, "r", encoding="utf8") as file:
        count={}
        prev_label=0
        data=[]
        for line in file:
            tokens = line.split()
            label = int(tokens[-1])
            if(label!= 0):
                if(prev_label!=label and prev_label!=0):
                    all_data_x.append(data)
                    all_data_y.append(prev_label)
                    count[prev_label]=len(data)
                           
                columns=len(tokens)-1
                if(prev_label !=label):
                    data+=[tokens[:-1]]
                prev_label=label
            
        # print(count)
        all_data_x.append(data)
        all_data_y.append(prev_label)
        count[prev_label]=len(data)
        for value in count.values():
            if(value>max_len):
                max_len=value
                

print the maximum length

In [67]:
print("maximun lenth of data =",max_len)

maximun lenth of data = 12


In [68]:
max_column=23 ## using question
num_labels=12
data_count=10*num_labels

In [69]:
def get_em_data(data,max_len,max_column):
    embedding_data= np.zeros((len(data), max_len, max_column))
    
    for index in range(len(data)):
        num_times=len(data[index])
        for i in range(num_times):
            embedding_data[index][i]= np.array(data[index][i], dtype=float)
    return embedding_data

Convert into numpy array

In [70]:
y_array=np.array(all_data_y)

print a sample array

In [71]:
y_array

array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  5,  5,  1,  2,  3,  4,
        6,  7,  8, 10, 11, 12,  9,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11,
       12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,
        4,  6,  7,  8,  9, 10, 11, 12,  5,  1,  2,  3,  4,  6,  7,  8,  9,
       10, 11, 12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12,  1,
        3,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4,  6,  7,
        8,  9, 10, 11, 12,  5,  5,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11,
       12])

In [72]:
x_array=get_em_data(all_data_x,max_len +1,max_column)

convert to categorical

In [73]:
y_convert =y_array-1
y_array_c = to_categorical(y_convert)


create a model

In [74]:
model = Sequential()

In [75]:
print(max_column)

23


This is the model SimpleRNN

In [76]:
model.add(SimpleRNN(units= 320 ,return_sequences=True,input_shape=(max_len+1,max_column)))
model.add(Dropout(0.25))
model.add(SimpleRNN(units= 400,return_sequences=True))
# model.add(Dropout(0.25))
model.add(SimpleRNN(units= 100))
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.25))
#model.add(Dense(units=64,activation='relu'))
model.add(Dense(units=num_labels,activation='softmax'))

###SimpleRNN
###Bidirectional(GRU)
#model.add(GRU(units= 128,return_sequences=True)

Define losss function

In [77]:
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics=['accuracy'])

Fit the model

In [78]:
result=model.fit(x_array,y_array_c,epochs=20,batch_size=20,validation_split=0.2, shuffle=True)
#batch_size=20,validation_split=0.2

Epoch 1/20
5/5 [==============================] - 3s 132ms/step - loss: 2.9037 - accuracy: 0.0417 - val_loss: 2.6054 - val_accuracy: 0.0833
Epoch 2/20
5/5 [==============================] - 0s 39ms/step - loss: 2.7348 - accuracy: 0.0521 - val_loss: 2.5617 - val_accuracy: 0.0833
Epoch 3/20
5/5 [==============================] - 0s 35ms/step - loss: 2.6516 - accuracy: 0.0625 - val_loss: 2.5453 - val_accuracy: 0.0833
Epoch 4/20
5/5 [==============================] - 0s 41ms/step - loss: 2.6411 - accuracy: 0.0833 - val_loss: 2.5490 - val_accuracy: 0.0833
Epoch 5/20
5/5 [==============================] - 0s 40ms/step - loss: 2.6397 - accuracy: 0.0521 - val_loss: 2.5450 - val_accuracy: 0.0833
Epoch 6/20
5/5 [==============================] - 0s 36ms/step - loss: 2.6336 - accuracy: 0.0625 - val_loss: 2.5537 - val_accuracy: 0.0833
Epoch 7/20
5/5 [==============================] - 0s 36ms/step - loss: 2.6297 - accuracy: 0.0625 - val_loss: 2.5637 - val_accuracy: 0.0833
Epoch 8/20
5/5 [==========

Predict some values

In [79]:
predicted = model.predict(x_array[2:8])

1/1 [==============================] - 0s 434ms/step


In [80]:
predicted[0]

array([0.06136221, 0.07104018, 0.07640945, 0.08456101, 0.10861189,
       0.08344835, 0.07355946, 0.08630357, 0.0665158 , 0.11349642,
       0.09018833, 0.08450334], dtype=float32)

In [81]:
predicted_labels = np.argmax(predicted, axis=1)

Find the accuracy

In [82]:
training_accuracy = result.history['accuracy']
validation_accuracy = result.history['val_accuracy']


In [83]:
print( "final training accuracy = ",training_accuracy[-1])
print( "final testing accuracy = ",validation_accuracy[-1])

final training accuracy =  0.09375
final testing accuracy =  0.0833333358168602


In [84]:
print( "maximum training accuracy = ",max(training_accuracy))
print( "maximum testing accuracy = ",max(validation_accuracy))

maximum training accuracy =  0.125
maximum testing accuracy =  0.0833333358168602


Save the model

In [44]:
model.save("SimpleRNN_model_3.h5")